In [7]:
import numpy as np
import matplotlib.pyplot as plt
import glob, importlib
from natsort import natsorted
import bead_utils as bu
import scipy.signal as sp

Script to analyze the changes in charge vs time from Jiaxiang's data

In [8]:
data_path = "/Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/jiaxiang_data/20230803/1/"

In [87]:
importlib.reload(bu)
flist = natsorted(glob.glob(data_path + "*.h5"))

## first make a correlation template
cdat, attr, _ = bu.get_data(flist[0])
template = bu.correlation_template(cdat, attr, make_plots=False)

charge_vec = []

for i,f in enumerate(flist):

    if(i%10 == 0): print("working on file %d: %s"%(i, f))
    
    cdat, attr, _ = bu.get_data(f)

    #bu.plot_raw_data(cdat, attr, nfft=2**19, do_psd=True, do_filt=True)
    
    fname = "corr_%d.png"%i
    corr_val = bu.plot_correlation_with_drive(cdat, template, attr, skip_drive=True)

    time = attr["Time"]

    charge_vec.append([time, corr_val])

np.save("charge_steps.npy", charge_vec)

charge_vec = np.array(charge_vec)

plt.figure()
plt.plot(charge_vec[:,0], charge_vec[:,1])
plt.show()

working on file 0: /Users/dcmoore/Library/CloudStorage/GoogleDrive-david.c.moore@yale.edu/My Drive/yale/uspheres/jiaxiang_data/20230803/1/LPmbar_xyzcool_0.h5


TypeError: list.append() takes exactly one argument (2 given)

In [7]:
attr["Fsamp"]

10000.0